# Demo of the Weather Logger Agent

In this notebook, we'll see how to write and use a very simple Weather Logger agent. It will support two operations: an observation that checks the current weather, and an action that logs the weather in a file. 

Let's start with various imports we need.


In [1]:
# Make a file `.env` and put your OPEN_AI_KEY there.

import os
from dotenv import load_dotenv
load_dotenv()
OPEN_AI_KEY = os.environ.get('OPEN_AI_KEY')

In [2]:
import requests
import openai
import json
from toolfuse import Tool, action, observation

openai.api_key = OPEN_AI_KEY
client = openai.OpenAI(api_key=OPEN_AI_KEY)


Now, let's define the Weather Logger Tool. It's a simple implementation of the `Tool` class, with two methods: `log` and `check`.

`check` goes to `wttr.in` and returns the weather in a human-readable format. `log` takes any message and appends it to a file.

In [3]:
class WeatherLogger(Tool):
    def __init__(self):
        super().__init__()
        self.log_file = "weather.txt"

    @action
    def log(self, message: str):
        """Logs a message to the log file."""
        with open(self.log_file, "a") as file:
            file.write("***\n" + message + "\n")

    @observation
    def weather(self, location: str) -> str:
        """Checks the current weather from the internet using wttr.in."""
        weather_api_url = f"http://wttr.in/{location}?format=%l:+%C+%t"
        try:
            response = requests.get(weather_api_url)
            return response.text
        except requests.exceptions.HTTPError as err:
            return f"HTTP Error: {err}"
        except requests.exceptions.RequestException as e:
            return f"Error: Could not retrieve weather data. {e}"
        
    def close(self):
        """Close the WeatherLogger tool and release any resources."""
        # Since there are no resources to release in this simple example, pass is used.
        pass


Let's check the weather:

In [4]:
tool = WeatherLogger()

In [5]:
message = tool.weather("Paris")
message

'Paris: Partly cloudy +11°C'

In [6]:
tool.log(message)
with open(tool.log_file, "r") as file:
    print(file.read())

***
Paris: Partly cloudy +11°C



Does it work? Super. However, we don't wrap the weather API call into a Tool just for the sake of it. 

We can now use it with AI. How? A Tool can export the list of supported actions and observations to a JSON schema and also execute actions from JSONs and dicts. 

Let's look at the schema.

In [7]:
schema = tool.json_schema()
schema

[{'name': 'log',
  'parameters': {'type': 'object',
   'properties': {'message': {'type': 'string'}},
   'required': ['message']},
  'description': 'Logs a message to the log file.'},
 {'name': 'weather',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string'}},
   'required': ['location']},
  'description': 'Checks the current weather from the internet using wttr.in.'}]

As you can see, the schema lists all actions we have with their parameters and descriptions. 

Given this information, we can implement a simple scenario:

* We inform the AI about the tools (in our case, `weather` and `log`) that it can use.
* We give it a task to check and describe weather in a given city.
* We start the loop: ask it for the next step, execute the next step, give it the results.

First things first. As you know, prompting is new programming, so a correct prompt is super important. Let's define the system prompt and the task for our little weather agent.

In [8]:
system_message = f"""You are a helpful poetical AI. 
You can check the weather, write a short poem to describe it and log all of that. 
You have the following tools awailable to you: 

{schema}

When you respond, always give a JSON with a correct tool called. For example:

{{"tool": "weather", "parameters": {{"location": "Thessaloniki"}}}}

{{"tool": "log", "parameters": {{"message": "The weather is nice."}}}}

Only answer with JSON. Avoid wrapping it in quotes.
"""

task = "Check the weather in Paris, describe it, and log the result."

Now, we define a couple of helper functions. One will send a question to AI and return the response, while the other one will use this response to execute the next step. 

In [9]:
def ask_ai(messages):
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=messages,
        max_tokens=1000
    )
    print(f"Full response: {response}")
    assistant_reply = response.choices[0].message.content
    print(f"Actual reply: {assistant_reply}")
    return assistant_reply

def execute_step(reply, tool):
    jdict = json.loads(reply)
    print(f"Parsed JSON: {jdict}")
    action = tool.find_action(jdict['tool'])
    print(f"Found action: {action.name}")
    result = tool.use(action, **jdict['parameters'])
    print(f"Result: {result}")
    return result

Now, we prepare the first messages we'll send to the Completion OpenAI API.

In [10]:
messages = []
messages.append({"role": "system", "content": system_message})
messages.append({"role": "user", "content": task})

And now, go! In the first loop, our little agent should suggest to check the weather.

In [11]:
reply = ask_ai(messages)
messages.append({"role": "assistant", "content": reply})
result = execute_step(reply, tool)
messages.append({"role": "user", "content": result})

Full response: ChatCompletion(id='chatcmpl-99VnOohk6ZIC2eYU4KcOjiFTxQt5W', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"tool": "weather", "parameters": {"location": "Paris"}}', role='assistant', function_call=None, tool_calls=None))], created=1712054202, model='gpt-4-1106-preview', object='chat.completion', system_fingerprint='fp_d986a8d1ba', usage=CompletionUsage(completion_tokens=15, prompt_tokens=134, total_tokens=149))
Actual reply: {"tool": "weather", "parameters": {"location": "Paris"}}
Parsed JSON: {'tool': 'weather', 'parameters': {'location': 'Paris'}}
Found action: weather
Result: Paris: Partly cloudy +11°C


In the second loop, it gets the weather, comes up with a little poem and uses `log` tool to record it.

In [12]:
reply = ask_ai(messages)
messages.append({"role": "assistant", "content": reply})
result = execute_step(reply, tool)

Full response: ChatCompletion(id='chatcmpl-99VnQsPQaKe8pnam7TipWUKnlrNSb', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"tool": "log", "parameters": {"message": "A sky of artists\' delight, in Paris so fair,\\nPartly clad in cloudy attire, with a cool, gentle air,\\nEleven degrees holds the day in its embrace,\\nWhile sun and clouds in a dainty dance do chase."}}', role='assistant', function_call=None, tool_calls=None))], created=1712054204, model='gpt-4-1106-preview', object='chat.completion', system_fingerprint='fp_e467c31c3d', usage=CompletionUsage(completion_tokens=65, prompt_tokens=165, total_tokens=230))
Actual reply: {"tool": "log", "parameters": {"message": "A sky of artists' delight, in Paris so fair,\nPartly clad in cloudy attire, with a cool, gentle air,\nEleven degrees holds the day in its embrace,\nWhile sun and clouds in a dainty dance do chase."}}
Parsed JSON: {'tool': 'log', 'parameters': {'message': "A sky of art

And here we are, with a beautiful poem about the weather in the city of your choice!

In [13]:
with open(tool.log_file, "r") as file:
    print(file.read())

***
Paris: Partly cloudy +11°C
***
A sky of artists' delight, in Paris so fair,
Partly clad in cloudy attire, with a cool, gentle air,
Eleven degrees holds the day in its embrace,
While sun and clouds in a dainty dance do chase.

